In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import pycaret.classification as pc

from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm, cv
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

# urls 400

In [2]:
df = dd.read_parquet('..\\full_data\\figma_plan\\full_gender_data_txt.parquet').compute()

# берем нужные колонки url
df = df.loc[:,df.columns[12:-1].to_list()+['user_id']]
top400 = np.load('top400_features.npy',allow_pickle=True)
df = df.loc[:,(df.columns.isin(top400))|(df.columns.isin(['user_id']))]
# добавляем target, удаляем nan
df = df.loc[:,(df.columns.isin(top400))|(df.columns.isin(['user_id']))]
df = df.merge(pd.read_parquet('..\\full_data\\public_train.pqt',columns=['user_id','is_male'])[['is_male','user_id']], on = 'user_id', how = 'inner')
df = df.loc[~(df['is_male'].isna()) & (df['is_male'] != 'NA')]
df['is_male'] = df['is_male'].astype('int8')

display(df.head())
display(df.info())

,googleads.g.doubleclick.net,yandex.ru,i.ytimg.com,vk.com,avatars.mds.yandex.net,ad.mail.ru,yastatic.net,apple.com,instagram.com,ads.adfox.ru,...,manrule.ru,programma-peredach.com,mcpehub.org,go64.ru,nail-styles.ru,other_url,other_female_urls_frac,other_male_urls_frac,user_id,is_male
0,0.196891,0.031088,0.025907,0.041451,0.046632,0.031088,0.051813,0.0,0.000000,0.025907,...,0.0,0.0,0.0,0.0,0.0,0.031088,0.005181,0.000000,0,0
1,0.072588,0.112703,0.040115,0.023878,0.104107,0.034384,0.081184,0.0,0.000000,0.019102,...,0.0,0.0,0.0,0.0,0.0,0.013372,0.001910,0.000955,1,0
2,0.092457,0.046229,0.126521,0.141119,0.072993,0.038929,0.046229,0.0,0.000000,0.026764,...,0.0,0.0,0.0,0.0,0.0,0.002433,0.000000,0.000000,2,0
3,0.021818,0.061818,0.007273,0.178182,0.094545,0.080000,0.069091,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010909,0.003636,0.003636,3,1
4,0.061776,0.030888,0.048906,0.083655,0.046332,0.061776,0.045045,0.0,0.003861,0.028314,...,0.0,0.0,0.0,0.0,0.0,0.011583,0.003861,0.000000,4,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 264326 entries, 0 to 269999
Columns: 402 entries, googleads.g.doubleclick.net to is_male
dtypes: float32(400), int32(1), int8(1)
memory usage: 406.6 MB


None

## train

In [3]:
x_train = df.drop(['is_male','user_id'],axis=1)
y_train = df['is_male']

pool_train = Pool(x_train, y_train,
                  cat_features = list(x_train.select_dtypes(include=['object']).columns))

model = CatBoostClassifier(iterations=2000,
                           learning_rate = 0.01,
                           random_strength = 1,
                           l2_leaf_reg = 8,
                           random_state=42,
                           task_type='GPU')

model.fit(pool_train, verbose=False,plot=True)
print(2 * roc_auc_score(y_train, model.predict_proba(x_train)[:,1]) - 1)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0.7289354188693253


In [ ]:
model.save_model('catboost_gender_urls')

In [4]:
model = CatBoostClassifier()
model.load_model('catboost_gender_urls')
print(2 * roc_auc_score(y_train, model.predict_proba(x_train)[:,1]) - 1)

0.7289353568885151


## test

In [2]:
df = dd.read_parquet('..\\full_data\\figma_plan\\full_gender_data_txt.parquet').compute()

#берем нужные колонки url
df = df.loc[:,df.columns[12:-1].to_list()+['user_id']]
top400 = np.load('top400_features.npy',allow_pickle=True)
df = df.loc[:,(df.columns.isin(top400))|(df.columns.isin(['user_id']))]

#оставляем id для теста
submit = dd.read_parquet('..\\full_data\\submit_2.pqt').compute()
submit = submit.sort_values('user_id', ignore_index=True)
test_df = submit.merge(df, on='user_id',how ='left')

In [4]:
model = CatBoostClassifier()
model.load_model('catboost_gender_urls')
display(model.predict_proba(test_df)[:,1])

array([0.20734899, 0.71953531, 0.17580539, ..., 0.46747215, 0.48323137,
       0.48212971])

In [6]:
submit['is_male'] = model.predict_proba(test_df)[:,1]
submit['age'] = pd.Series(data=-1,index=submit.index)

In [7]:
submit.to_csv('top400_male.csv', index=False)

# full

In [14]:
df = dd.read_parquet('..\\full_data\\ready_gender_data.parquet').compute()

# добавляем target, удаляем nan
df = df.merge(pd.read_parquet('..\\full_data\\public_train.pqt',columns=['user_id','is_male'])[['is_male','user_id']], on = 'user_id', how = 'inner')
df = df.loc[~(df['is_male'].isna()) & (df['is_male'] != 'NA')]
df['is_male'] = df['is_male'].astype('int8')

display(df.head())
display(df.info())

,user_id,region_name,male_fraction_region,city_name,city_count,cpe_manufacturer_name,price,holyday_fraction,morning_fraction,day_fraction,...,127.0.0.1,bukvaprava.ru,ngs24-ru.turbopages.org,forum.mfd.ru,other_url,other_female_urls_frac,other_male_urls_frac,male_probability_by_urls,catboost_top400urls,is_male
0,0,Москва,0.461904,Москва,1,Samsung,2990.0,0.243523,0.119,0.554,...,0.0,0.0,0.0,0.0,0.031088,0.005181,0.000000,0.500491,0.076492,0
1,1,Москва,0.461904,Москва,6,Xiaomi,NaN,0.200573,0.323,0.347,...,0.0,0.0,0.0,0.0,0.013372,0.001910,0.000955,0.523982,0.105959,0
2,2,Республика Коми,0.472075,Печора,1,Huawei,5915.0,0.299270,0.187,0.482,...,0.0,0.0,0.0,0.0,0.002433,0.000000,0.000000,0.525968,0.458972,0
3,3,Воронежская область,0.458936,Воронеж,1,Huawei Device Company Limited,13990.0,0.181818,0.178,0.353,...,0.0,0.0,0.0,0.0,0.010909,0.003636,0.003636,0.519823,0.359567,1
4,4,Краснодарский край,0.463641,Анапа,9,Huawei,12990.0,0.326898,0.372,0.349,...,0.0,0.0,0.0,0.0,0.011583,0.003861,0.000000,0.510070,0.054218,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 264326 entries, 0 to 269999
Columns: 1102 entries, user_id to is_male
dtypes: float32(1096), int32(1), int8(2), object(3)
memory usage: 1.1+ GB


None

In [15]:
x_train = df.drop(['is_male','user_id'],axis=1)
y_train = df['is_male']

pool_train = Pool(x_train, y_train,
                  cat_features = list(x_train.select_dtypes(include=['object']).columns))

model = CatBoostClassifier(iterations=25000,
                           learning_rate = 0.01,
                           random_state=42,
                           task_type='GPU')

model.fit(pool_train, verbose=100)
print(2 * roc_auc_score(y_train, model.predict_proba(x_train)[:,1]) - 1)

0:	learn: 0.6876687	total: 781ms	remaining: 5h 25m 12s
100:	learn: 0.4806422	total: 5.55s	remaining: 22m 49s
200:	learn: 0.4546239	total: 10.1s	remaining: 20m 43s
300:	learn: 0.4479074	total: 14.6s	remaining: 19m 55s
400:	learn: 0.4444166	total: 19s	remaining: 19m 23s
500:	learn: 0.4422319	total: 23.3s	remaining: 18m 57s
600:	learn: 0.4403895	total: 27.4s	remaining: 18m 33s
700:	learn: 0.4387856	total: 31.6s	remaining: 18m 13s
800:	learn: 0.4373641	total: 35.7s	remaining: 17m 58s
900:	learn: 0.4360148	total: 39.9s	remaining: 17m 47s
1000:	learn: 0.4348266	total: 44s	remaining: 17m 35s
1100:	learn: 0.4337396	total: 48s	remaining: 17m 21s
1200:	learn: 0.4327140	total: 52s	remaining: 17m 9s
1300:	learn: 0.4316908	total: 56s	remaining: 16m 59s
1400:	learn: 0.4307214	total: 59.9s	remaining: 16m 49s
1500:	learn: 0.4298137	total: 1m 3s	remaining: 16m 40s
1600:	learn: 0.4289335	total: 1m 7s	remaining: 16m 31s
1700:	learn: 0.4281002	total: 1m 11s	remaining: 16m 22s
1800:	learn: 0.4273078	total:

## test

In [16]:
df = dd.read_parquet('..\\full_data\\ready_gender_data.parquet').compute()

#оставляем id для теста
submit = dd.read_parquet('..\\full_data\\submit_2.pqt').compute()
submit = submit.sort_values('user_id', ignore_index=True)
test_df = submit.merge(df, on='user_id',how ='left')

display(df.head())
display(df.info())

,user_id,region_name,male_fraction_region,city_name,city_count,cpe_manufacturer_name,price,holyday_fraction,morning_fraction,day_fraction,...,megapteka-ru.turbopages.org,127.0.0.1,bukvaprava.ru,ngs24-ru.turbopages.org,forum.mfd.ru,other_url,other_female_urls_frac,other_male_urls_frac,male_probability_by_urls,catboost_top400urls
0,0,Москва,0.461904,Москва,1,Samsung,2990.0,0.243523,0.119,0.554,...,0.0,0.0,0.0,0.0,0.0,0.031088,0.005181,0.000000,0.500491,0.076492
1,1,Москва,0.461904,Москва,6,Xiaomi,NaN,0.200573,0.323,0.347,...,0.0,0.0,0.0,0.0,0.0,0.013372,0.001910,0.000955,0.523982,0.105959
2,2,Республика Коми,0.472075,Печора,1,Huawei,5915.0,0.299270,0.187,0.482,...,0.0,0.0,0.0,0.0,0.0,0.002433,0.000000,0.000000,0.525968,0.458972
3,3,Воронежская область,0.458936,Воронеж,1,Huawei Device Company Limited,13990.0,0.181818,0.178,0.353,...,0.0,0.0,0.0,0.0,0.0,0.010909,0.003636,0.003636,0.519823,0.359567
4,4,Краснодарский край,0.463641,Анапа,9,Huawei,12990.0,0.326898,0.372,0.349,...,0.0,0.0,0.0,0.0,0.0,0.011583,0.003861,0.000000,0.510070,0.054218


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Columns: 1101 entries, user_id to catboost_top400urls
dtypes: float32(1096), int32(1), int8(1), object(3)
memory usage: 1.7+ GB


None

In [17]:
model.predict_proba(test_df.drop('user_id',axis=1))[:,1]

array([0.21077311, 0.90167581, 0.07017782, ..., 0.49237029, 0.41178107,
       0.6048454 ])

In [18]:
submit['is_male'] = model.predict_proba(test_df.drop('user_id',axis=1))[:,1]
submit['age'] = pd.Series(data=-1,index=submit.index)

In [19]:
submit.to_csv('gender_predict25000.csv', index=False)